In [10]:
import pandas as pd
import time
import datetime as dt
import numpy as np

In [11]:
! pip install praw

In [12]:
! pip install psaw

In [13]:
import praw
import time
from psaw import PushshiftAPI # hmm may not work let's see
import os

In [14]:
from tqdm import tqdm

In [15]:
! pip install pmaw

I'm using the PRAW library to access the reddit data. It is built for Reddit and is easier to use than the requests library. 
https://praw.readthedocs.io/en/latest/getting_started/quick_start.html 

^ This is where the documentation is located for PRAW

Reddit API Guidelines:

- 60 requests per minute (1 request per second)
- Access 100 submissions (posts) at a time
- This means that $60 * 100 = 6000$ submissions can be accessed per minute (36,000 per hour!)

`reddit.subredit('sub').new()` (or any other method that accesses submissions) will return a `ListingGenerator` object which only requests for the submission when iterated through (so we can put a pause using time in the for loop to meet the API requests per minute guidelines)

Comments are stored as a `CommentForest` instances

- Each instance is an iterable
- `list()` method on comments will flatten and return all the comments
- `list()` function will return the top comments from the post, based on the sorting method
-  `submission.comment_sort = 'new'` , for example, will sort the comments based on recency 

Comments are automatically sorted by 'confidence', which is from an algorithm that arranges comments based on how relevant and useful they are to the discussion. 
--> 
***This sorting algorithm takes into account factors such as the number of upvotes and downvotes a comment has received, as well as the reputation of the users who have voted on the comment.***

___Comments that have a high number of upvotes and few or no downvotes from users who are known to be reputable contributors to the subreddit will be given a higher confidence score and will appear at the top of the comment section. Conversely, comments that have a low number of upvotes or a high number of downvotes from users who have a history of making low-quality contributions will be given a lower confidence score and will be pushed further down the comment section.___

***Sorting comments by confidence is designed to make it easier for readers to find high-quality and informative comments that are most relevant to the discussion, while filtering out low-quality or off-topic comments***

Out of 200 posts (and going through the newest posts) the farthest back in date time we go is: 2023-04-03 22:40:17
- How many iterations will we need to get a decent amount of data?? lol
- this should take a while to run!

Going back 3000 got us 992 rows in the data frame (??) and the furthest back date time is '2023-03-19 06:21:16'

Scraper made for accessing posts by date and creating CSV's by date.

In [16]:
import requests

In [33]:
before_sub = submissions_df['created_utc'].min()

In [34]:
# before = int(dt.datetime(2021,12,15,0,0).timestamp())
# after = int(dt.datetime(2020,12,1,0,0).timestamp())
selected_columns = ['id', 'title', 'selftext', 'url',
                    'score',
                   'created_utc', 'over_18']


submissions_df = pd.DataFrame(columns = selected_columns)



url = 'https://api.pushshift.io/reddit/search/submission'
before = before_sub
for i in tqdm(range(100)):
    search_params = {'subreddit' : 'UCSD', 
                 'size': 1_000,
                 'before': before
    }
    response = requests.get(url, search_params)

    data = response.json()['data']

    data_frame = pd.DataFrame(data)
    
    submissions_df = pd.concat((data_frame[selected_columns], submissions_df), axis=0)
    
    before = int(data_frame['created_utc'].min())
    time.sleep(1)

start = dt.datetime.fromtimestamp(submissions_df['created_utc'].min())
end = dt.datetime.fromtimestamp(submissions_df['created_utc'].max())
print('start date:', start)
print('end date:', end)

submissions_df.to_csv(path_or_buf='subs' + str(start).split()[0] + 'to' + str(end).split()[0] + '.csv')

submissions_start = start
submissions_end = end

 14%|█▍        | 14/100 [01:26<08:49,  6.15s/it]


KeyError: "None of [Index(['id', 'title', 'selftext', 'url', 'score', 'created_utc', 'over_18'], dtype='object')] are in the [columns]"

In [35]:

start = dt.datetime.fromtimestamp(submissions_df['created_utc'].min())
end = dt.datetime.fromtimestamp(submissions_df['created_utc'].max())
print('start date:', start)
print('end date:', end)

submissions_df.to_csv(path_or_buf='subs' + str(start).split()[0] + 'to' + str(end).split()[0] + '.csv')

start date: 2010-10-13 01:37:56
end date: 2017-03-13 03:08:45


Just eliminate the t3_ from `link_id` and that's the submission id

In [28]:
# get data back to march 2017 (matching the submissions data)
# Make comments data frame
selected_columns = ['id', 'body', 'score', 'link_id', 
                   'controversiality', 'created_utc', 'nest_level']
data_frame = None
comments_df = pd.DataFrame(columns=selected_columns)
url = 'https://api.pushshift.io/reddit/search/comment'
before = before_saved # time.time()
for i in tqdm(range(200)):
    search_params = {'subreddit' : 'UCSD', 
                 'size': 1_000,
                 'before': before
    }
    response = requests.get(url, search_params)

    data = response.json()['data']

    data_frame = pd.DataFrame(data)
    
    comments_df = pd.concat((data_frame[selected_columns], comments_df), axis=0)
    
    before = int(data_frame['created_utc'].min())
    time.sleep(1)

start = dt.datetime.fromtimestamp(comments_df['created_utc'].min())
end = dt.datetime.fromtimestamp(comments_df['created_utc'].max())
print('start date:', start)
print('end date:', end)

comments_df.to_csv(path_or_buf='comments' + str(start).split()[0] + 'to' + str(end).split()[0] + '.csv')

 93%|█████████▎| 186/200 [12:57<00:58,  4.18s/it]


KeyError: "None of [Index(['id', 'body', 'score', 'link_id', 'controversiality', 'created_utc',\n       'nest_level'],\n      dtype='object')] are in the [columns]"

In [27]:
before_saved = before

In [29]:
start = dt.datetime.fromtimestamp(comments_df['created_utc'].min())
end = dt.datetime.fromtimestamp(comments_df['created_utc'].max())
print('start date:', start)
print('end date:', end)

comments_df.to_csv(path_or_buf='comments' + str(start).split()[0] + 'to' + str(end).split()[0] + '.csv')

start date: 2010-10-13 06:28:08
end date: 2018-12-16 00:37:41


In [21]:
comments_df.shape

(200707, 7)

### Comment Iterations
The number of iterations in the above for loop and how long it goes back:
- 10: 3/19/23 to 4/9/23 --> 4 weeks
- 20: 2/27/23 to 4/9/23 --> 8 weeks
- 10 iterations ~ 4 weeks!

need to go back 317 weeks if my calculations are right (to 2017-03-11)